# 3. 데이터 전처리 이해와 실무
## 3.1 데이터 변환 : 구간화 정규화
### 데이터 변환 (Transformation)
- 여러 형태로 표현된 데이터 값을 다양한 분석 방법론에 적용하기 위해 원시 형태에서 다른 형식으로 바꾸는 과정  
=> 주어진 목적 기반의 올바른 결과 획득을 위하여 원시 데이터를 **데이터 분석에 용이하도록 형태 변환**
#### 변환 목적 및 특징
- 빠른 특성 파악 : 데이터의 특성을 빠르게 파악 가능, 파생변수 생성 및 단순화를 통한 결과 리포팅 등 활용 (ex: 연령 -> 연령대)
- 분석 알고리즘 적용 : 변수 간 범위가 다를 경우 알고리즘 적용 시 영향력의 차이가 발생함, 올바른 학습을 위해 다른 범위를 지닌 변수들의 변환 필요
### 구간화(Binning)
- 연속형 데이터를 구간으로 구별하여 범주화 형태 변환
1. 지정 길이 기반 구간 정의
    - 사용자 기준으로 데이터 범위의 간격을 구분하여 관측치를 나누는 방안
2. 분포 기반 구간 정의
    - 관측치가 각 구간 내 동일한 개수로 구분되도록 나누는 방안

### 정규화(Normalization)
- 데이터 탐색 및 기계학습 적용을 위한 연속형 변수 변환
1. 최대-최소 정규화
    - 데이터 구간을 0에서 1사이로 변환
    - 특정 데이터의 위치 파악  
    `x^~ = (x - min(x)) / (max(x) - min(x))`
2. Z-점수 정규화 (표준화)
    - 0을 중심으로 양쪽으로 데이터 분포시킴
    - 특정 데이터가 평균과 얼마나 떨어져 있는지 파악
## 실습
데이터 필요

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
click_data = pd.read_csv("./data/click_sample_data.csv", encoding='cp949')
click_data.head(15)

In [ ]:
click_data.info()

click_copy = click_data.copy()

### 구간화(Binnig)
구간화 방안
1. 지정 길이 기반 구간 : 데이터 범위를 사용자의 기준대로 정의하고 구간화
2. 분포 기반 구간 : 각 데이터가 모든 구간 내 동일한 개수로 구분되도록 구간화

In [ ]:
click_copy.describe()

- 기사 클릭 수가 20회인 기사와 10회인 기사 비교
- 20회 클릭 된 기사가 2배 더 흥미있는 기사로 정의 할 수 있는가?  
-> 없음. 기사의 카테고리, 저널 등 클릭을 유도하는 여러 요소가 있기 때문

- 따라서, 구간화를 통해 새로운 의미를 지닌 변수로 변환이 필요하며 목적 설정이 중요함
- 클릭 횟수를 기준으로 범주로 변환하여 각 범주별 의미 설정 가능 (도메인 지식 검토 필요)
#### 1. 지정 길이 기반 구간화
클릭 수 기반으로 기사 관심도 정의 기반 범주형 변환 (3개 구간 설정)

In [ ]:
bins = [0, 100, 500, np.max(click_copy['num_click'])]
names = ["low", "medium", "high"]

click_copy['pref'] = pd.cut(click_copy['num_click'], bins, names)

In [ ]:
click_copy['pref'].value_counts()

In [ ]:
click_copy.head()

#### 2. 분포 기반 구간화
qcut의 경우 동일한 관측치가 배치되도록 구간을 분할

In [ ]:
n = 3
click_copy['pref_qcut'] = pd.qcut(click_copy['num_click'], 3, labels=names)
print(click_copy['pref_qcut'].value_counts())

**만일 qcut 실행 시 주로 발생하는 오류**
- 데이터가 극단치의 Skewed 한 경우 발생
- 데이터의 각 구간은 동일할 수 없기 때문
    - [1, 1, 1, 1, 1, 3, 8, 10] 이라는 구간은 생성 불가능
- 그러한 경우에는 분포 기반 구간화(qcut) 보다 사용자 기준 기반 범주화 혹은 구간 개수 변화 고려

### 정규화
**기계학습 알고리즘의 각 변수 별 영향력의 차이를 조정할 필요 존재함**
- 각 변수 별 범위가 다를 경우 학습 안정성이 떨어질 수 있으므로 간격 조정 필요
#### 1. 최대-최소 정규화 : 변수 안의 값을 [0, 1] 구간의 값을 갖도록 구성하고 데이터 군 내에서 특정 데이터가 가지는 위치를 보고자 할 때 사용

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

click_copy = click_data.copy()

click_copy['minmax_values'] = scaler.fit_transform(click_copy[['num_click']])
click_copy.head()

In [ ]:
click_copy.describe()

#### 2. Z-Score 정규화 : 특정 데이터가 평균을 기준으로 얼마나 떨어져 있는지 파악하고 측정 스케일이 다른 경우의 데이터를 비교하는데 사용

In [ ]:
from sklearn.preprocessing import StandardScaler
std_scaler = StandardScaler()

click_copy['std_values'] = std_scaler.fit_transform(click_copy[['num_click']])
click_copy.head()

In [ ]:
click_copy.describe()

**어느 방안이 더 나은 것인가?**
- 상황에 따라 달라지며, 정규화 수행과 수행하지 않은 결과의 차이가 큰 것이 중요한 포인트임
- 연속형 데이터를 기계학습 모델링에 적용하기 위해서 변수 간의 상대적 크기 차이를 제거할 필요가 반드시 존재함
- 예시)
    - 대표적 비지도학습인 군집분석 중 Kmeans는 수치형 변수로 적용하는 알고리즘
    - 변수간 수치걱 distance(거리)를 연산하여 유사한 집단으로 묶는 알고리즘
    - 변수가 각자 다른 범위를 가진 경우 유사 집단 구성 시, 변수 간 영향력의 차이가 반영되어 올바르지 못한 결과 도출 가능성 존재